# LBA: Grocery store prices
In this assignment, we model the cost of groceries in different parts of the world. To what extent do
grocery prices vary by country and store brand? Are grocery prices and the geographical
distribution of different grocery stores correlated with other cost-of-living measures- for example,
rent and real estate prices?

## Task 1: Prices
What is the basic average price for each product? You need to think carefully about how to
anchor the basic price for each product since this will depend on the currency used as well
as the distribution of prices.<br><br>

- Anchor in Euros
- Anchor at units that make sense for each category but is base 10 metric, e.g. milk in l, apples and potatoes in kg, meat and butter in 0.1kg. 

I tried several python packages for currency conversion but none had all of the currencies I wanted and some were unclear on which day's conversion rate they were using so I pivoted to using Mathematica.

In [7]:
# testing my currencyconversion setup
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr
session = WolframLanguageSession()
session.evaluate(wlexpr('CurrencyConvert[Quantity[13.25, "MAD"], "EUR"][[1]]'))

1.2296597100794315

In [9]:
import numpy as np
from string import Template

### Processing done before import
- changed country column to be standardized:
    - USA, UK, Germany (got rid of cities and alternative names for the countries)
    - Also change supermarket names to make sure the same market would have the same name there (e.g. 4 people went to the 4th street Trader Joe's)
- turned all rent prices into one number 
    - where necessary, looked up the price myself to be consistent in source within the city
    - San Francisco: https://www.zumper.com/blog/map-san-francisco-neighborhood-rent-prices-fall-2018/
    - Berlin: https://www.immobilienscout24.de/content/dam/is24/ibw/dokumente/mietmap_berlin_2020.jpg (this is the 2020 version of the 2016 map from the instructions)
- verified outliers with the authors, for example the 73GBP for bananas should be 0.73GBP

In [10]:
# I downloaded the data as tsv to avoid commas in people's responses
data = np.loadtxt("LBAData.tsv", delimiter="\t",dtype=object)

In [11]:
#dimensions
print(np.shape(data))
#example datapoint
print(data[4])

(65, 68)
['10/23/2020 18:40:00' 'katja.dellalibera@minerva.kgi.edu'
 'Katja Della Libera' 'Germany' 'EUR'
 'Alnatura Super Natur Markt, Münzgasse 4A, 78462 Konstanz'
 'Luxury (expensive)' '1000' '1' '3.99' '1' '3.99' '1' '3.99' '1' '2.29'
 '' '' '' '' '0.1' '0.79' '1' '5.99' '0.1' '0.99' '1' '2.49' '2' '4.99'
 '2' '5.99' '1' '1.49' '1' '0.95' '1' '1.49' '1' '3.99' '1' '5.99' '1'
 '7.69' '1' '1.49' '1' '1.29' '1' '1.15' '0.25' '2.99' '0.25' '2.49'
 '0.25' '2.59' '10' '4.29' '10' '1.99' '1' '0.45' '1' '32.9' '' '' '' '']


In [12]:
# make a dictionary to encode all the indexes
products = ['apples','bananas','tomatoes','potatoes','flour','rice','milk','butter','eggs','chicken']
indexes = {'country': 3,
          'currency': 4,
          'category': 6,
          'rent': 7}

for no, prod in enumerate(products):
    for i in range (3):
        indexes[prod + str(i)] = 8+6*no+i*2
print(indexes)

{'country': 3, 'currency': 4, 'category': 6, 'rent': 7, 'apples0': 8, 'apples1': 10, 'apples2': 12, 'bananas0': 14, 'bananas1': 16, 'bananas2': 18, 'tomatoes0': 20, 'tomatoes1': 22, 'tomatoes2': 24, 'potatoes0': 26, 'potatoes1': 28, 'potatoes2': 30, 'flour0': 32, 'flour1': 34, 'flour2': 36, 'rice0': 38, 'rice1': 40, 'rice2': 42, 'milk0': 44, 'milk1': 46, 'milk2': 48, 'butter0': 50, 'butter1': 52, 'butter2': 54, 'eggs0': 56, 'eggs1': 58, 'eggs2': 60, 'chicken0': 62, 'chicken1': 64, 'chicken2': 66}


In [31]:
# show all the quantities for one product
data[:,indexes['bananas0']]

array(['Product 1 quantity (kg)', '1', '1', '1', '1', '1', '1', '1', '1',
       '0.9', '0.118', '0.118', '0.45', '1', '1', '0.118', '1', '1',
       '0.4535924', '0.453592', '0.54', '1', '1', '1', '1', '1', '1', '1',
       '1', '1', '1', '1', '1', '0.6', '0.6', '0.78', '1', '1', '0.91',
       '1', '0.118', '1', '1', '1', '1', '0.12', '0.45', '0.45', '0.2',
       '0.5', '1', '3.5', '1', '0.4536', '1.36078', '1.3608', '0.453592',
       '1.36078', '0.453592', '1', '1', '1', '1', '1', '1'], dtype=object)

In [58]:
# write a function that normalizes by target unit and converts the currency
def get_normalized_price(i, target_unit):
    units = np.array([1 if x=="" else x for x in data[1:,i]], dtype=np.float)
    price = np.array([0 if x=="" else x for x in data[1:,i+1]], dtype=np.float)
    normalized_price = price/units*target_unit
    normalized_price_EUR = []
    for no, x in enumerate(normalized_price):
        original_currency = str(data[no+1,indexes['currency']])
        t = Template('CurrencyConvert[Quantity[${price}, "${origin}"], "EUR"][[1]]')
        new_price = session.evaluate(wlexpr(t.substitute(price=x,origin = original_currency)))
        normalized_price_EUR.append(new_price)
    return normalized_price_EUR

In [64]:
apples = np.transpose([get_normalized_price(indexes['apples0'],1),
           get_normalized_price(indexes['apples1'],1),
           get_normalized_price(indexes['apples2'],1)])
bananas = np.transpose([get_normalized_price(indexes['bananas0'],1),
           get_normalized_price(indexes['bananas1'],1),
           get_normalized_price(indexes['bananas2'],1)])
tomatoes = np.transpose([get_normalized_price(indexes['tomatoes0'],1),
           get_normalized_price(indexes['tomatoes1'],1),
           get_normalized_price(indexes['tomatoes2'],1)])
potatoes = np.transpose([get_normalized_price(indexes['potatoes0'],1),
           get_normalized_price(indexes['potatoes1'],1),
           get_normalized_price(indexes['potatoes2'],1)])
flour = np.transpose([get_normalized_price(indexes['flour0'],1),
           get_normalized_price(indexes['flour1'],1),
           get_normalized_price(indexes['flour2'],1)])
rice = np.transpose([get_normalized_price(indexes['rice0'],1),
           get_normalized_price(indexes['rice1'],1),
           get_normalized_price(indexes['rice2'],1)])
milk = np.transpose([get_normalized_price(indexes['milk0'],1),
           get_normalized_price(indexes['milk1'],1),
           get_normalized_price(indexes['milk2'],1)])
# measure butter in half-pounds (250g)
butter = np.transpose([get_normalized_price(indexes['butter0'],0.25),
           get_normalized_price(indexes['butter1'],0.25),
           get_normalized_price(indexes['butter2'],0.25)])
# get 10 eggs, the common unit in Germany (so EUR)
eggs = np.transpose([get_normalized_price(indexes['eggs0'],10),
           get_normalized_price(indexes['eggs1'],10),
           get_normalized_price(indexes['eggs2'],10)])
# get 0.5kg of chicken
chicken = np.transpose([get_normalized_price(indexes['chicken0'],0.5),
           get_normalized_price(indexes['chicken1'],0.5),
           get_normalized_price(indexes['chicken2'],0.5)])

KeyboardInterrupt: 

In [ ]:
bananas[9]

In [ ]:
# use a masked array since some values will be 0 (the missing ones)
X = np.ma.masked_equal(X,0)

## Task 2: Factors influencing prices
How much does each of the following factors modify the basic price of the product (up or
down)?
- The geographical location (country) of the grocery store.
- Brand of the grocery store. Since we are getting data from multiple countries, you will need to specify whether the store brand is considered budget (cheap), mid-range, or luxury (expensive). This should be based on what you think the general public perception of the store brand is.
- Does price variation by geographical location correlate with variation in rental prices, or
not?

Explain in your report how strong each of these effects is. Which has the greatest influence
on price variation between shops?

Notes:
- Two levels of categorical variables
    - country
    - grocery store rating
- one numeric variable: rent price
- Determine significance and size of the variation between categories